In [ ]:
import json
import subprocess
import os

import altair as alt
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# alt.data_transformers.enable('csv')
alt.data_transformers.disable_max_rows()

In [ ]:
ips = [
    "34.67.69.87",
    "104.198.214.12",
]

In [ ]:
for i, ip in tqdm(enumerate(ips), total=len(ips)):
    path = f'../outputs/ss-ecc-portion/'
    if not os.path.exists(path):
        os.makedirs(path)

    process = subprocess.Popen(f'scp -r -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null chanwutk@{ip}:"/home/chanwutk/Documents/apperception/outputs/run/*" {path}', shell=True)
    process.wait()


In [ ]:
DATA_DIR = '../outputs/ss-ecc-portion'
runs = [f'/run-{i}' for i in range(len(ips))]

In [ ]:
failed_videos = {}
for f in os.listdir(DATA_DIR):
    if not f.startswith('failed_videos'):
        continue

    with open(os.path.join(DATA_DIR, f), 'r') as _f:
        content = json.load(_f)

    name = f[len('failed_videos--'):-len('.json')]
    failed_videos[name] = []
    for v_name, err in content:
        failed_videos[name].append(v_name)
failed_videos

In [ ]:
stage_order = [
    'InView', 'DecodeFrame', 'Detection2D.YoloDetection', 'Detection2D.ObjectTypeFilter',
    'DepthEstimation', 'Detection3D.FromDetection2DAndDepth', 'Detection3D.FromDetection2DAndRoad',
    'DetectionEstimation', 'Tracking.StrongSORT', 'Tracking2D.StrongSORT', 'StrongSORTCacheBenchmark', 'Tracking3D.FromTracking2DAndDepth',
    'Tracking3D.FromTracking2DAndRoad', 'SegmentTrajectory.FromTracking3D'
]

test_names = {
    'de': 'Only Detection Estimation',
    'noopt': 'Baseline',
    'inview': 'Only In-View',
    'geo': 'Only Geo Depth Estimation',
    'objectfilter': 'Only Object Filter',
    'opt': 'Optimized',
    'optde': 'Optimized with Detection Estimation'
}

test_order = [
    'noopt',
    'inview',
    'objectfilter',
    'geo',
    'de',
    'opt',
    'optde'
]

In [ ]:
def combine_perf():
    data = []
    for filename in os.listdir(DATA_DIR):
        if filename.startswith('perf'):
            test, run = filename.split("--")[1].split(".")[0].split("_")
            with open(os.path.join(DATA_DIR, filename), 'r') as f:
                for stage in json.load(f):
                    stagename = stage['stage']
                    for b in stage['benchmark']:
                        after, before = b['keep']
                        assert after <= before
                        skip = (1. - (after / before)) if before != 0 else 0
                        data.append({
                            **b,
                            'stage': stagename,
                            '_test': test,
                            'test': 'cache=' + test.split('-')[-1] + run,
                            # 'test_order': test_order.index(test),
                            'run': run,
                            'name': b['name'].split('/')[-1].split('.')[0],
                            'skip': skip,
                            'stage_order': stage_order.index(stagename)
                        })
    return data


perf = combine_perf()

In [ ]:
# with open('./output/perf.json', 'w') as f:
#     json.dump([p for p in perf if p['run'] == "0"], f, indent=1)

In [ ]:
len([p for p in perf if p['run'] == "0"])

In [ ]:
perf[0]

In [ ]:
for name, videos in failed_videos.items():
    if len(videos) != 0:
        print(name, videos)

In [ ]:
df_perf = pd.DataFrame.from_dict(perf)
# df_perf = df_perf[df_perf['run'] == "0"]
df_perf[:10]

In [ ]:
df_perf.groupby(['test', 'stage'])[['name']].count()

In [ ]:
(alt.Chart(df_perf)
    .mark_bar()
    .encode(
        x='average(runtime)',
        y=alt.Y('test:O', sort=alt.Sort({'field': 'test_order'})),
        color=alt.Color('stage:N', sort=stage_order),
        order=alt.Order('order:O')
    )
    .properties(height=300, width=800)
)

# Break-down runtime of StrongSORT

In [ ]:
def combine_ss_perf():
    data = []
    for filename in os.listdir(DATA_DIR):
        if filename.startswith('perf'):
            test, run = filename.split("--")[1].split(".")[0].split("_")
            with open(os.path.join(DATA_DIR, filename), 'r') as f:
                for stage in json.load(f):
                    stagename = stage['stage']
                    if stagename != 'Tracking.StrongSORT':
                        continue
                    for b in stage['ss-benchmark']:
                        # after, before = b['keep']
                        # assert after <= before
                        # skip = (1. - (after / before)) if before != 0 else 0
                        data.append({
                            'file': b['file'],
                            'portion': 'Load Data',
                            'runtime': b['load_data'],
                            '_test': test,
                            'order': 0,
                        })
                        data.append({
                            'file': b['file'],
                            'portion': 'Init',
                            'runtime': b['init'],
                            '_test': test,
                            'order': 1,
                        })
                        data.append({
                            'file': b['file'],
                            'portion': 'Update Camera',
                            'runtime': b['update_camera'],
                            '_test': test,
                            'order': 2,
                        })
                        data.append({
                            'file': b['file'],
                            'portion': 'Tracking',
                            'runtime': b['tracking'] - b['update_camera'],
                            '_test': test,
                            'order': 3,
                        })
                        data.append({
                            'file': b['file'],
                            'portion': 'Postprocessing',
                            'runtime': b['postprocess'],
                            '_test': test,
                            'order': 4,
                        })
                        # data.append({
                        #     **b,
                        #     'stage': stagename,
                        #     '_test': test,
                        #     'test': 'cache=' + test.split('-')[-1] + run,
                        #     # 'test_order': test_order.index(test),
                        #     'run': run,
                        #     'name': b['name'].split('/')[-1].split('.')[0],
                        #     'skip': skip,
                        #     'stage_order': stage_order.index(stagename)
                        # })
    return data


ss_perf = combine_ss_perf()

In [ ]:
df_ss_perf = pd.DataFrame.from_dict(ss_perf)
df_ss_perf

In [ ]:
(alt.Chart(df_ss_perf)
    .mark_bar()
    .encode(
        x='average(runtime)',
        y=alt.Y('_test:O'), #, sort=alt.Sort({'field': 'test_order'})),
        color=alt.Color('portion:N', sort=stage_order),
        order=alt.Order('order:O')
    )
    .properties(height=300, width=800)
)

# Compare Failed ECC

In [ ]:
def all_failed_src_dst(folder):
    failed_ecc = {}
    for file in sorted(os.listdir(os.path.join(DATA_DIR, 'failed-ecc', folder))):
        timestamp, _back = file.split(".jpg")[0].split("--", 1)
        end = 'src' if _back.endswith('src') else 'dst'
        _back = _back[:-len('--dst')]
        prefix = ''
        if _back.startswith('-'):
            prefix = '-'
            _back = _back[1:]

        src, dst = _back.split('-', 1)
        src = prefix + src

        # print(timestamp, src, dst, '           ', file)
        key = f'{src}/{dst}'

        if key not in failed_ecc:
            failed_ecc[key] = []
        failed_ecc[key].append(1)
    return failed_ecc


failed_ecc_0 = all_failed_src_dst('0')
failed_ecc_1 = all_failed_src_dst('1')

print("They should have the same number of src/dst to fail")
print(len(failed_ecc_0))
print(len(failed_ecc_1))

print("All of the failed src/dst for both run should be the same")
for fe0, fe1 in zip(sorted(failed_ecc_0.keys()), sorted(failed_ecc_1.keys())):
    print(fe0 == fe1, fe0, fe1)

# Compare if both runs have exactly the same output

# Both StrongSORT results should have the same files

In [ ]:
runs = {'true': [], 'false': []}
for run, files in runs.items():
    for file in os.listdir(os.path.join(DATA_DIR, f'sort--ss-cache-{run}_0')):
        files.append(file)

print('Size should be the same:')
print(len(runs['true']))
print(len(runs['false']))

print('Videos with different contents:')
for r0, r1 in zip(sorted(runs['false']), sorted(runs['true'])):
    if r0 != r1:
        print(r0, r1)

    with open(os.path.join(DATA_DIR, 'sort--ss-cache-false_0', r0), 'r') as f:
        content0 = f.read()

    with open(os.path.join(DATA_DIR, 'sort--ss-cache-true_0', r1), 'r') as f:
        content1 = f.read()

    if content0 != content1:
        print(r0, r1)

# Done: Confirmed that caching does not have negative effects on StrongSORT